# Addestramento Agente SARSA per Cyber Security

Questo notebook implementa l'addestramento di un agente SARSA per proteggere una rete da attacchi casuali.

## Come Funziona SARSA?
1. **State (S)**: Lo stato corrente della rete
2. **Action (A)**: L'azione di difesa scelta
3. **Reward (R)**: La ricompensa ricevuta
4. **State' (S')**: Il nuovo stato dopo l'azione
5. **Action' (A')**: La prossima azione che verrà eseguita

L'agente impara aggiornando i suoi valori Q basandosi sulla formula:
Q(S,A) = Q(S,A) + α[R + γQ(S',A') - Q(S,A)]

In [ ]:
pip install gym
pip install gym_idsgame
pip install numpy
pip install matplotlib
pip install seaborn
pip install tqdm

In [ ]:
# Importiamo le librerie necessarie
import gym
import gym_idsgame
import numpy as np
import matplotlib.pyplot as plt
from src.agents.sarsa_agent import SARSAAgent
from tqdm.notebook import tqdm

# Per i grafici più belli
import seaborn as sns
sns.set_style("whitegrid")

In [ ]:
# Configurazione dei parametri
CONFIG = {
    'num_episodes': 1000,      # Numero di episodi di training
    'max_steps': 500,          # Passi massimi per episodio
    'learning_rate': 0.001,    # Quanto velocemente l'agente impara
    'gamma': 0.99,             # Importanza delle ricompense future
    'epsilon': 1.0,            # Probabilità iniziale di esplorazione
    'epsilon_min': 0.01,       # Probabilità minima di esplorazione
    'epsilon_decay': 0.995     # Velocità di decadimento dell'esplorazione
}

In [ ]:
# Creazione dell'ambiente
env = gym.make("idsgame-random-attack-v0", num_layers=3, num_servers_per_layer=3)

# Stampiamo informazioni sull'ambiente
print("Informazioni sull'ambiente:")
print(f"Spazio degli stati: {env.observation_space}")
print(f"Spazio delle azioni: {env.action_space}")

In [ ]:
# Inizializzazione dell'agente
agent = SARSAAgent(
    state_size=env.observation_space.n,
    action_size=env.action_space.n,
    learning_rate=CONFIG['learning_rate'],
    gamma=CONFIG['gamma'],
    epsilon=CONFIG['epsilon'],
    epsilon_min=CONFIG['epsilon_min'],
    epsilon_decay=CONFIG['epsilon_decay']
)

In [ ]:
def train():
    """Funzione principale di training"""
    rewards_history = []    # Lista per tracciare le ricompense
    losses_history = []     # Lista per tracciare le losses
    
    # Loop principale di training
    for episode in tqdm(range(CONFIG['num_episodes']), desc='Training'):
        state = env.reset()  # Reset dell'ambiente
        total_reward = 0     # Ricompensa totale per questo episodio
        episode_losses = []  # Losses per questo episodio
        
        # Scegliamo la prima azione
        action = agent.get_action(state)
        
        # Loop per un singolo episodio
        for step in range(CONFIG['max_steps']):
            # Eseguiamo l'azione nell'ambiente
            next_state, reward, done, _ = env.step(action)
            
            # Scegliamo la prossima azione (questo è ciò che rende SARSA on-policy)
            next_action = agent.get_action(next_state)
            
            # Aggiorniamo l'agente e otteniamo la loss
            loss = agent.update(state, action, reward, next_state, next_action)
            
            # Aggiorniamo i valori per il prossimo step
            state = next_state
            action = next_action
            total_reward += reward
            episode_losses.append(loss)
            
            if done:
                break
        
        # Salviamo le statistiche dell'episodio
        rewards_history.append(total_reward)
        losses_history.append(np.mean(episode_losses))
        
        # Stampiamo le statistiche ogni 100 episodi
        if (episode + 1) % 100 == 0:
            print(f"\nEpisodio {episode + 1}")
            print(f"Ricompensa media ultimi 100 episodi: {np.mean(rewards_history[-100:]):.2f}")
            print(f"Epsilon corrente: {agent.epsilon:.3f}")
    
    return rewards_history, losses_history

In [ ]:
# Avviamo il training
print("Inizio training...")
rewards_history, losses_history = train()

In [ ]:
# Visualizziamo i risultati
plt.figure(figsize=(15, 5))

# Plot delle ricompense
plt.subplot(1, 2, 1)
plt.plot(rewards_history)
plt.title('Ricompense per Episodio')
plt.xlabel('Episodio')
plt.ylabel('Ricompensa Totale')

# Plot della loss
plt.subplot(1, 2, 2)
plt.plot(losses_history)
plt.title('Loss Media per Episodio')
plt.xlabel('Episodio')
plt.ylabel('Loss Media')

plt.tight_layout()
plt.show()

In [ ]:
# Salviamo il modello addestrato
agent.save('models/sarsa_agent.npy')
print("Modello salvato con successo!")

## Analisi dei Risultati

Dopo l'addestramento, possiamo osservare:
1. Come variano le ricompense nel tempo
2. Come diminuisce la loss (errore di previsione)
3. Come l'agente migliora le sue prestazioni

Alcuni aspetti da considerare:
- Un aumento delle ricompense indica che l'agente sta imparando
- Una diminuzione della loss indica che le previsioni migliorano
- L'epsilon che diminuisce indica meno esplorazione e più sfruttamento